In [1]:
from libpysal.weights import *
import xarray as xr
%reload_ext memory_profiler

In [2]:
da = xr.open_rasterio("nasadem_sd.tif")
da.shape

(1, 3515, 5510)

*Profiling sparse weight builders for raster data*

`lat2SW` is very performant since it creates regular lattice without any missing values to deal with. It straight away creates diagonals and offsets which are then shipped to dia matrix constructor.
Memory consumpttion is high due to use of `list` instead of `np.array`

In [3]:
# memory profiling vanilla lat2SW
%memit lat2SW(*da[0].shape, "queen")

peak memory: 4134.30 MiB, increment: 3859.86 MiB


In [4]:
# perf-testing vanilla lat2SW
%timeit lat2SW(*da[0].shape, "queen")

5.47 s ± 489 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Next is `da2WSP` which uses `lat2SW` to build sparse matrix and then through boolean indexing it removes missing rows and columns from the created matrix.
Performance takes a hit due to boolean indexing of csr matrix.

In [5]:
%memit da2WSP(da, "queen")

peak memory: 4208.72 MiB, increment: 3933.50 MiB


In [6]:
%timeit da2WSP(da, "queen")

7.18 s ± 72.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


To create matrix with only non missing values in mind, `lat2SW`/`dia_matrix` was out of question since it is higly oriented towards building regular lattice.

Next options were to build either [DOK](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.dok_matrix.html#scipy.sparse.dok_matrix), [CSR/CSC](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix), or [COO](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html#scipy.sparse.coo_matrix)...

- It was impossible (for me) to Numba-fy `DOK` builder due to its structure and without numba it was too slow.
- In case of `CSR/CSC` I was not able to think of a multithreaded implementation (still exploring a way to build this)
- From the start I was biased towards `COO` as it's structure is quite simple, can be numba-fied, farely easy to incorporate multithreading and fast conversion to `CSR/CSC` matrix

In [7]:
# memory profiling COO based da2WSP
%memit da2WSP2(da, "queen")

peak memory: 3121.64 MiB, increment: 2758.57 MiB


In [8]:
# perf-testing COO based da2WSP
%timeit da2WSP2(da, "queen")

4.13 s ± 284 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Still working on parallel implementation as this is only single threaded, right now it initializes row and col arrays each of dtype=uint32 (can hold a raster of size upto 65535x65535) 

In [ ]:
def da2WSP2(da, criterion="rook", band=None):
    band = da_checker(da, band)
    da = da[band-1:band]
    ser = da.to_series()
    mask = (ser != da.nodatavals[0]).to_numpy()
    ids = np.where(mask)[0]
    row, col, data = _swbuilder(*da[0].shape, ids, mask, criterion)
    n = len(ids)
    sw = sparse.coo_matrix((data, (row, col)), shape=(n, n), dtype=np.int8).tocsr()
    ser = ser[ser != da.nodatavals[0]]
    index = ser.index
    wsp = WSP(sw, index=index)
    return wsp


@njit
def _swbuilder(nrows, ncols, ids, mask, criterion):
    n = len(ids)
    mask = mask * 1
    mask[ids] = np.arange(len(ids), dtype=np.uint32)
    d = 4 if criterion == "rook" else 8
    rows = np.empty(d*n, dtype=np.uint32)
    cols = np.empty(d*n, dtype=np.uint32)
    j = 0
    for i in prange(n):
        k = ids[i]
        if ((k+1) % ncols) != 0:
            r = mask[k + 1]
            if r:
                rows[j], cols[j] = i, r
                j += 1
                rows[j], cols[j] = r, i
                j += 1
        if (k // ncols) < (nrows - 1):
            r = mask[k+ncols]
            if r:
                rows[j], cols[j] = i, r
                j += 1
                rows[j], cols[j] = r, i
                j += 1
        if criterion == "queen":
            if (k // ncols) < (nrows - 1):
                if (k % ncols) != 0:
                    r = mask[k+ncols-1]
                    if r:
                        rows[j], cols[j] = i, r
                        j += 1
                        rows[j], cols[j] = r, i
                        j += 1
                if ((k+1) % ncols) != 0:
                    r = mask[k+ncols+1]
                    if r:
                        rows[j], cols[j] = i, r
                        j += 1
                        rows[j], cols[j] = r, i
                        j += 1
    rows = rows[:j]
    cols = cols[:j]
    data = np.ones(j, dtype=np.int8)
    return rows, cols, data